In [6]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 79.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.probability import FreqDist
from nltk.util import ngrams
import pandas as pd

from gensim import corpora
import gensim

import ast
import fs

In [10]:
INTERIM_FILES_DIR = fs.open_fs("MCD-NLP-HPT/data/interim")
INTERIM_COMMENTS_CLEANED_DIR = INTERIM_FILES_DIR.getsyspath("Comments_cleaned.csv")

In [17]:
df = pd.read_csv(INTERIM_COMMENTS_CLEANED_DIR)
df.head()

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full,Tokens,Tokens_without_stopwords,Tokens_without_stopwords_stemmed,Tokens_lemmatized,Tokens_without_stopwords_lemmatized
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"['Me', 'gusta', 'su', 'podcast', ',', 'pero', ...","['Me', 'gusta', 'su', 'podcast', 'pero', 'solo...","['Me', 'gusta', 'podcast', 'solo', 'escuchado'...","['me', 'gust', 'podcast', 'sol', 'escuch', 'yo...","['yo', 'gustar', 'su', 'podcast', ',', 'pero',...","['yo', 'gustar', 'podcast', 'sólo', 'escuchar'..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,['weeeeey'],['weeeeey'],['weeeeey'],['weeeeey'],['weeeeey'],['weeeeey']
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"['Les', 'voy', 'a', 'enviar', 'una', 'foto', '...","['Les', 'voy', 'a', 'enviar', 'una', 'foto', '...","['Les', 'voy', 'enviar', 'foto', 'TARAPACA']","['les', 'voy', 'envi', 'fot', 'tarapac']","['él', 'ir', 'a', 'enviar', 'uno', 'foto', 'de...","['él', 'ir', 'enviar', 'foto', 'tarapaca']"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"['Entonces', 'Teca', 'esta', 'solteroooooooooo...","['Entonces', 'Teca', 'esta', 'solteroooooooooo...","['Entonces', 'Teca', 'solterooooooooooooooo', ...","['entonc', 'tec', 'solteroooooooooooooo', 'woo...","['entonces', 'teca', 'estar', 'solterooooooooo...","['entonces', 'teca', 'solterooooooooooooooo', ..."
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"['Me', 'encantan', 'sus', 'podcast', ',', 'los...","['Me', 'encantan', 'sus', 'podcast', 'los', 'e...","['Me', 'encantan', 'podcast', 'escucho', 'spot...","['me', 'encant', 'podcast', 'escuch', 'spotify...","['yo', 'encantar', 'su', 'podcast', ',', 'él',...","['yo', 'encantar', 'podcast', 'escuchar', 'spo..."


In [20]:
category_map = {
    1: "Quejas o sugerencias de mejora",
    2: "Propuestas y recomendaciones",
    3: "Experiencias personales",
    4: "Correcciones o datos adicionales",
    5: "Felicitaciones, agradecimientos y saludos",
    6: "Comentarios humorísticos o memes",
    7: "Comentarios generales"
}

In [19]:
df_corpus = df.groupby("category_id")["Tokens_without_stopwords_lemmatized"].apply(
    lambda x: [token for sublist in x for token in ast.literal_eval(sublist)]
)

In [21]:
df_corpus = pd.DataFrame(df_corpus).reset_index()
df_corpus['category_description'] = df_corpus['category_id'].map(category_map)
df_corpus

,category_id,Tokens_without_stopwords_lemmatized,category_description
0,1,"[yo, gustar, podcast, sólo, escuchar, youtube,...",Quejas o sugerencias de mejora
1,2,"[ir, llegar, podcast, guerra, china, vs, taiwa...",Propuestas y recomendaciones
2,3,"[el, capítulo, 98, micro, historia, gracia, no...",Experiencias personales
3,4,"[tarapacá, región, chile, entender, enredar, d...",Correcciones o datos adicionales
4,5,"[yo, encantar, podcast, escuchar, spotify, via...","Felicitaciones, agradecimientos y saludos"
5,6,"[entonces, teca, solterooooooooooooooo, wooooo...",Comentarios humorísticos o memes
6,7,"[weeeeey, él, ir, enviar, foto, tarapaca, prof...",Comentarios generales


In [17]:
# df_video_corpus['List_of_Tokens_combined'][0]

In [22]:
# text = ["The movie was very scary and long.", "The Movie was not scary,","The movie was good"]
corpus = df_corpus['Tokens_without_stopwords_lemmatized'][1]
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(corpus)
features = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df_dense = pd.DataFrame(denselist, columns=features)
df_dense

,007,100,101,1500,2018,250,28,40,490,600,...,zizka,zona,zunzu,zunzunegui,árabe,él,étnico,íker,únansar,único
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
def get_category_words_top(top, dataframe):
    """
    Obtiene las palabras principales por categoría ordenadas por frecuencia.

    Args:
        top (int): Número de palabras principales a seleccionar.
        dataframe (pd.DataFrame): DataFrame con las palabras y sus frecuencias.

    Returns:
        pd.DataFrame: DataFrame con las palabras principales por categoría.
    """
    dataframe = dataframe.T.sort_index(axis=1, ascending=True)
    columns_data = {}

    for col_index in range(dataframe.shape[1]):
        dataframe = dataframe.reset_index()
        dataframe.drop(columns='level_0', inplace=True, errors='ignore')
        dataframe.sort_values(by=[col_index], inplace=True, ascending=False)

        # Seleccionar las primeras `top` palabras para cada categoría
        columns_data[col_index] = [dataframe['index'].iloc[:top].tolist()]

    # Crear el DataFrame final de una vez usando `pd.concat`
    category = pd.DataFrame(columns_data)
    return category

In [33]:
Category_Words = get_category_words_top(3,df_dense)
Category_Words

,0,1,2,3,4,5,6,7,8,9,...,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130
0,"[ir, 007, parte]","[llegar, ir, aburrido]","[podcast, llegar, parque]","[guerra, podcast, abracitooo]","[china, guerra, parecer]","[vs, china, 718]","[taiwan, paralelo, pacquiao]","[guerra, taiwan, 600]","[medio, guerra, vs]","[oriente, medio, 490]",...,"[enseñar, anime, faltar]","[hacer, enseñar, imaginación]","[proceso, hacer, teotihuacan]","[cero, proceso, doctor]","[bien, tiempo, teotihuacan]","[v3rg4s, bien, super]","[v3rg4s, embargo, vídeo]","[capítulo, v3rg4s, dalessio]","[hacer, capítulo, jaja]","[hierro, arango, dedicar]"


In [37]:
# df_corpus['Tokens_without_stopwords_lemmatized'][1]

In [51]:
documents = [df_corpus['Tokens_without_stopwords_lemmatized'][3]]

# Crear el diccionario
dictionary = corpora.Dictionary(documents)

# Ver el diccionario (opcional)
print(dictionary.token2id)  # Muestra la correspondencia palabra-ID

{'...': 0, '0': 1, '007': 2, '1': 3, '10': 4, '100': 5, '1094': 6, '12:30': 7, '13': 8, '15': 9, '150': 10, '16': 11, '17': 12, '180': 13, '1825': 14, '1864': 15, '1869': 16, '1880': 17, '1895': 18, '1920': 19, '1945': 20, '1950': 21, '1984': 22, '1:07': 23, '1:11': 24, '1:12': 25, '2': 26, '20': 27, '2015': 28, '2019': 29, '2020': 30, '21:40': 31, '23': 32, '23:16': 33, '24:12': 34, '250': 35, '25:03': 36, '27:30': 37, '29:29': 38, '3': 39, '300': 40, '30:02': 41, '30:58': 42, '31': 43, '33': 44, '35': 45, '350': 46, '365': 47, '39': 48, '3:31': 49, '4': 50, '420': 51, '44': 52, '45:31': 53, '48:05': 54, '5': 55, '50': 56, '509': 57, '52': 58, '58:44': 59, '59:41': 60, '5:26': 61, '5:54': 62, '6': 63, '68': 64, '6:25': 65, '7': 66, '73': 67, '80': 68, '8:55': 69, '9': 70, '90': 71, 'ARGENTINA': 72, 'Abelardo': 73, 'Alianza': 74, 'Antonio': 75, 'Argelinos': 76, 'Augusto': 77, 'BLUES': 78, 'BRUTAL': 79, 'Banda': 80, 'Belen': 81, 'Belice': 82, 'Blanco': 83, 'Bolivia': 84, 'Bond': 85, 'Br

In [52]:
# Crear el corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Ver el corpus (opcional)
print(corpus)  # Ejemplo: [(0, 1), (1, 1)] -> palabras con su frecuencia

[[(0, 16), (1, 3), (2, 1), (3, 9), (4, 1), (5, 5), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 6), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 5), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 3), (67, 1), (68, 3), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 3), (85, 1), (86, 7), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 3), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 8), (101, 2), (102, 2), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 2), (109, 2), (110, 1

In [53]:
123@gmail.com123@gmail.comfrom gensim.models import LdaModel

# Crear el modelo LDA
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=5)

# Ver los tópicos generados
topics = lda_model.print_topics(num_words=4)  # Muestra las 4 palabras más representativas de cada tópico
for topic in topics:
    print(topic)

(0, '0.009*"si" + 0.008*"el" + 0.007*"decir" + 0.007*"hacer"')
(1, '0.004*"si" + 0.004*"decir" + 0.003*"el" + 0.003*"hacer"')
(2, '0.002*"si" + 0.002*"el" + 0.001*"decir" + 0.001*"él"')
(3, '0.001*"el" + 0.001*"poder" + 0.001*"si" + 0.001*"hacer"')


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2820e488-6f1b-466d-af14-a66826f012e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>